<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Create an instance of ImageDataGenerator with augmentation options
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)
datagen.fit(x_train_cnn)

# Create a validation set from the training data
x_train_cnn, x_val_cnn = x_train_cnn[:-12000], x_train_cnn[-12000:]
y_train, y_val = y_train[:-12000], y_train[-12000:]

# Load a pre-trained VGG16 model without the top layer (the classification part)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
model_transfer = models.Sequential()
model_transfer.add(layers.UpSampling2D(size=(8, 8), input_shape=(28, 28, 1)))  # Resizing input to match VGG16
model_transfer.add(layers.Conv2D(3, (3, 3), padding='same'))  # Matching channel number to 3
model_transfer.add(base_model)
model_transfer.add(layers.Flatten())
model_transfer.add(layers.Dense(256, activation='relu'))
model_transfer.add(layers.Dense(10, activation='softmax'))

# Compile and train the new model
model_transfer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_transfer.fit(x_train_cnn, y_train, epochs=5, validation_data=(x_val_cnn, y_val))

# Evaluate the model
test_loss_transfer, test_acc_transfer = model_transfer.evaluate(x_test_cnn, y_test)
print(f'Test accuracy: {test_acc_transfer}')